In [ ]:
!pip install pandas azure-storage-blob pyodbc


In [1]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
import io
from datetime import datetime 
import pyodbc
import os
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

load_dotenv()


# Configura tu conexión al Blob Storage de Azure
AZURE_API_KEY = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY =  os.getenv("OPENAI_API_KEY")
usuario_email_mercadona = os.getenv('USUARIO_MERCADONA')
contrasena_mercadona = os.getenv('CONTRASENA_MERCADONA')

def get_blob_client(nombre_archivo):
    connect_str = AZURE_API_KEY
    container_name = "scrapingmercadona"
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=nombre_archivo)
    return blob_client

# Conexión a la base de datos SQL de Azure
#server = 'scrapin.database.windows.net'
server = os.getenv('DB_SERVER')
database = os.getenv('DB_NAME')
username = os.getenv('DB_USER')
password = os.getenv('DB_PWD')
driver = '{ODBC Driver 17 for SQL Server}'

database_to_check = database

connection_string = f'DRIVER={driver};SERVER={server};UID={username};PWD={password}'

fecha_hoy = datetime.now().strftime("%Y-%m-%d")
blob_name = f"{fecha_hoy}_productos.csv"


In [ ]:

# Conectar al servidor (sin especificar la base de datos para realizar la verificación)
conn = pyodbc.connect(connection_string, autocommit=True)
cursor = conn.cursor()
conn

In [ ]:


# Lista de archivos CSV en el blob

blob_client = get_blob_client(blob_name)
# Leer los CSV desde el blob y concatenarlos en un DataFrame

dfs = []
csv_data = blob_client.download_blob().readall()
df = pd.read_csv(io.StringIO(csv_data.decode('latin1')), sep=',', on_bad_lines='skip')


# Concatenar todos los DataFrames
all_data = df

# Renombrar las columnas manualmente

all_data.columns = ['Categoria','Subcategoria','Subsubcategoria','Fecha','ID','Nombre','Precio']

# Convertir la columna 'Precio' a float, reemplazando valores no válidos por NaN
all_data['Precio'] = pd.to_numeric(all_data['Precio'], errors='coerce')
#all_data['Precio'] = all_data['Precio'].astype(int)
#all_data['Precio'] = all_data['Precio'].round(2)


# Eliminar filas con valores NaN en la columna 'Precio'
all_data = all_data.dropna(subset=['Precio'])

# Convertir la columna 'Fecha' a formato datetime
all_data['Fecha'] = pd.to_datetime(all_data['Fecha'], format='%Y-%m-%d')

all_data['Producto'] = all_data['Nombre'].copy()
all_data.drop('Nombre',axis=1,inplace=True)

#all_data.to_excel('mercadown.xlsx')

In [ ]:
columns_to_fill = ['Categoria', 'Subcategoria', 'Subsubcategoria', 'Producto']
all_data[columns_to_fill] = all_data[columns_to_fill].fillna(value='Ninguno')


In [ ]:


cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+password)
cursor = cnxn.cursor()

tabla = 'ProductosPreciosMercadona'
create_table_query = f'''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='{tabla}' AND xtype='U')
CREATE TABLE {tabla} (
    Categoria NVARCHAR(255),
    Subcategoria NVARCHAR(255),
    Subsubcategoria NVARCHAR(255),
    Fecha DATE,
    ID int,
    Precio FLOAT,
    Producto NVARCHAR(255),
)
'''

cursor.execute(create_table_query)
cnxn.commit()

# Insertar los datos en la tabla
for index, row in all_data.iterrows():
    cursor.execute('''
    INSERT INTO ProductosPreciosMercadona (Categoria, Subcategoria, Subsubcategoria, Fecha, ID, Precio, Producto)
    VALUES (?, ?, ?, ?, ?, ?,?)
    ''', row['Categoria'], row['Subcategoria'], row['Subsubcategoria'], row['Fecha'], row['ID'], row['Precio'],row['Producto'])


cnxn.commit()
cursor.close()
cnxn.close()


In [ ]:
all_data.replace